# Projeto 2 - Ciência dos Dados

Nome: Ellen Beatriz Shen

Nome: Gabriela Moreno Boriero

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [ ]:
pip install emoji

In [2]:
import math
import os.path
import pandas as pd
import json
from random import shuffle
import re 
import emoji
from emoji import UNICODE_EMOJI

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [3]:
#Leitura do excel para treinar o código
tudo = pd.read_excel("base_acai.xlsx", sheet_name="Treinamento")

#Série de todos os dados + sua tabela absoluta
serie_tudo = pd.Series(tudo.Treinamento)
tabela_tudo_absoluta = serie_tudo.value_counts()

In [4]:
#Função para limpar os sinais básicos.
def limpeza(texto):
    import string
    pontuacao = '[!-.:?;]'
    padrao = re.compile(pontuacao)
    text_subbed = re.sub(padrao, ' ', texto)
    letras=[]
    for letra in text_subbed:  
        letras.append(letra)
    i=1
    espaço=" "
    frase_separando_emoji=""
    while i<= (len(letras)-1):        
        if letras[i-1]==letras[i]:
            if letras[i] in UNICODE_EMOJI:
                frase_separando_emoji+=letras[i-1]
                frase_separando_emoji+=(espaço)
                frase_separando_emoji+=(letras[i])
                letras.remove(letras[i])
            i+=1
        else:
            frase_separando_emoji+=(letras[i-1])
            i+=1
    return frase_separando_emoji

In [5]:
#
relevante_acai = limpeza((open("relevante.txt", "r", encoding="utf8").read()).lower())
irrelevante_acai = limpeza((open("irrelevante.txt", "r", encoding="utf8").read()).lower())

In [6]:
relevante_relativa = (pd.Series(relevante_acai.split())).value_counts(True)
relevante_absoluta = (pd.Series(relevante_acai.split())).value_counts(False)

irrelevante_relativa = (pd.Series(irrelevante_acai.split())).value_counts(True)
irrelevante_absoluta = (pd.Series(irrelevante_acai.split())).value_counts(False)

In [7]:
probabilidade_relevante = relevante_absoluta.sum()/tabela_tudo_absoluta.sum()
probabilidade_irrelevante = irrelevante_absoluta.sum()/tabela_tudo_absoluta.sum()

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
#sem laplace

In [23]:
base_teste = pd.read_excel("base_acai.xlsx",sheet_name="Teste")
series_tweets = pd.Series(list(base_teste.Teste))

tweets_limpos = []
for tweet in series_tweets:
    tweets_limpos.append(limpeza(tweet.lower()))

    
classificacao = {}
classificado=[]
for frase in tweets_limpos:
    palavras = list(frase.split())
    for palavra in palavras:
        relevante = 1
        n_relevante = 1
        if palavra in relevante_relativa:
            relevante *= relevante_relativa[palavra]
        if palavra in irrelevante_relativa:
            n_relevante *= irrelevante_relativa[palavra]
        else:
            relevante*1
            n_relevante*1
    prob_relevante = probabilidade_relevante*relevante
    prob_n_relevante = probabilidade_irrelevante*n_relevante
    if prob_relevante > prob_n_relevante:
        classificacao[frase] = 0
        classificado.append(0)
    else:
        classificacao[frase] = 1
        classificado.append(1)

        
teste_classificado = pd.Series(classificacao)

In [24]:
#comparando o teste com a classificação
for tweet in classificacao:
    data = {
    'Frase': tweets_limpos  ,
    'Resultado Teste': classificado,
    'Classificação': base_teste["Classe"]
    }
tabela_classificacao = pd.DataFrame(data, columns=['Frase', 'Resultado Teste','Classificação'])


In [26]:
certos=0
erros=0
for teste, certo in zip(classificado, base_teste["Classe"]):
    if teste==certo:
        certos+=1
    else:
        erros+=1

porcentagem_certos=(certos/229)*100
porcentagem_erros=(erros/229)*100
print("Porcentagem de acertos= {}%".format(porcentagem_certos))
print("Porcentagem de erros= {}%".format(porcentagem_erros))

Porcentagem de acertos= 63.75545851528385%
Porcentagem de erros= 67.24890829694323%


In [27]:
verdadeiros_positivo=0
falsos_positivos=0
verdadeiros_negativos=0
falsos_negativos=0
#relevante=0 e irrelevante=1
for resultado_teste,resultado_certo in zip(classificado,base_teste["Classe"]):
    if resultado_certo==0 and resultado_teste==0:
        verdadeiros_positivo+=1
    elif resultado_certo==1 and resultado_teste==0:
        falsos_positivos+=1
    elif resultado_certo==1 and resultado_teste==1:
        verdadeiros_negativos+=1
    elif falsos_negativos==0 and resultado_teste==1:
        falsos_negativos+=1
    
Porcentagem_verdadeiros_positivo=(verdadeiros_positivo/229)*100
Porcentagem_falsos_positivos=(falsos_positivos/229)*100
Porcentagem_verdadeiros_negativos=(verdadeiros_negativos/229)*100
Porcentagem_falsos_negativos=(falsos_negativos/229)*100

print("Porcentagem de verdadeiros positivos: {}%".format(Porcentagem_verdadeiros_positivo))
print("Porcentagem de falsos positivos: {}%".format(Porcentagem_falsos_positivos))
print("Porcentagem de verdadeiros negativos: {}%".format(Porcentagem_verdadeiros_negativos))
print("Porcentagem de falsos negativos: {}%".format(Porcentagem_falsos_negativos))



Porcentagem de verdadeiros positivos: 56.33187772925764%
Porcentagem de falsos positivos: 58.951965065502186%
Porcentagem de verdadeiros negativos: 7.423580786026202%
Porcentagem de falsos negativos: 0.43668122270742354%


In [8]:
def laplace(palavra, tabela1, tabela2):
    if palavra in tabela1:
        x = tabela1[palavra]
    else:
        x = 0
        
    if palavra in tabela2:
        y = tabela2[palavra]
    else:
        y = 0 
    
    prob_relevante = (x + 1)/ (len(tabela1) + len(set(tabela1+tabela2)))
    prob_irrelevante = (y + 1)/ (len(tabela2) + len(set(tabela1+tabela2)))
    return [prob_relevante, prob_irrelevante]    

In [9]:
base_teste = pd.read_excel("base_acai.xlsx",sheet_name="Teste")
series_tweets = pd.Series(list(base_teste.Teste))

In [10]:
tweets_limpos = []
for tweet in series_tweets:
    tweets_limpos.append(limpeza(tweet.lower()))

    
classificacao = {}
lista_classificador=[]
for frase in tweets_limpos:
    palavras_da_frase = list(frase.split())
    relevante = 1
    irrelevante = 1
    
    for palavra in palavras_da_frase:

        if palavra in relevante_relativa and \
        palavra in irrelevante_relativa:
            relevante *= relevante_relativa[palavra]
            irrelevante *= irrelevante_relativa[palavra]
            
        elif palavra in relevante_relativa and \
        palavra not in irrelevante_relativa:
            relevante *= relevante_relativa[palavra]
            irrelevante *= laplace(palavra,relevante_relativa,irrelevante_relativa)[1]
        
        elif palavra in irrelevante_relativa and \
        palavra not in relevante_relativa:
            relevante *= laplace(palavra,relevante_relativa,irrelevante_relativa)[0]
            irrelevante *= irrelevante_relativa[palavra]
            
        else:
            relevante *= laplace(palavra,relevante_relativa,irrelevante_relativa)[0]
            irrelevante *= laplace(palavra,relevante_relativa,irrelevante_relativa)[1]
            
    prob_relevante = probabilidade_relevante*relevante
    prob_irrelevante = probabilidade_irrelevante*irrelevante
    
    if prob_relevante > prob_irrelevante:
        classificacao[frase] = 0
        lista_classificador.append(0)
    else:
        classificacao[frase] = 1
        lista_classificador.append(1)
       
classificados = pd.Series(classificacao)

In [11]:
#comparando o teste com a classificação 
for tweet in classificacao:
    dic={
        'Tweet': tweets_limpos,
        'Resultafo Teste': lista_classificador,
        'Classificação': base_teste["Classe"]
    }
tabela_classifiação= pd.DataFrame(dic, columns=['Tweet','Resultafo Teste','Classificação'])

In [13]:
certos=0
errados=0

for teste,certo in zip(lista_classificador,base_teste["Classe"]):
    if teste==certo:
        certos+=1
    else:
        errados+=1
       
porcentagem_certos=(certos/299)*100
porcentagem_errados=(errados/299)*100
print("Porcentagem de acertos= {}%".format(porcentagem_certos))
print("Porcentagem de erros= {}%".format(porcentagem_errados))

Porcentagem de acertos= 62.87625418060201%
Porcentagem de erros= 37.45819397993311%


In [14]:
verdadeiros_positivo=0
falsos_positivos=0
verdadeiros_negativos=0
falsos_negativos=0
#relevante=0 e irrelevante=1
for resultado_teste,resultado_certo in zip(lista_classificador,base_teste["Classe"]):
    if resultado_certo==0 and resultado_teste==0:
        verdadeiros_positivo+=1
    elif resultado_certo==1 and resultado_teste==0:
        falsos_positivos+=1
    elif resultado_certo==1 and resultado_teste==1:
        verdadeiros_negativos+=1
    elif falsos_negativos==0 and resultado_teste==1:
        falsos_negativos+=1
    
Porcentagem_verdadeiros_positivo=(verdadeiros_positivo/229)*100
Porcentagem_falsos_positivos=(falsos_positivos/229)*100
Porcentagem_verdadeiros_negativos=(verdadeiros_negativos/229)*100
Porcentagem_falsos_negativos=(falsos_negativos/229)*100

print("Porcentagem de verdadeiros positivos: {}%".format(Porcentagem_verdadeiros_positivo))
print("Porcentagem de falsos positivos: {}%".format(Porcentagem_falsos_positivos))
print("Porcentagem de verdadeiros negativos: {}%".format(Porcentagem_verdadeiros_negativos))
print("Porcentagem de falsos negativos: {}%".format(Porcentagem_falsos_negativos))



Porcentagem de verdadeiros positivos: 56.76855895196506%
Porcentagem de falsos positivos: 41.04803493449782%
Porcentagem de verdadeiros negativos: 25.327510917030565%
Porcentagem de falsos negativos: 0.43668122270742354%


In [31]:
por_acerts_sem_laplace=62.87625418060201
por_V_P_sem_laplace= 56.76855895196506
por_F_P_sem_laplace= 41.04803493449782
por_V_N_sem_laplace= 25.327510917030565
por_F_N_sem_laplace= 0.43668122270742354

porcentagem_certos_sem_separar_emoji= 61.87290969899666
Porcentagem_verdadeiros_positivo_sem_separar_emoji= 55.895196506550214
Porcentagem_falsos_positivos_sem_separar_emoji= 41.48471615720524
Porcentagem_verdadeiros_negativos_sem_separar_emoji= 24.890829694323145
Porcentagem_falsos_negativos_sem_separar_emoji= 0.43668122270742354

data = {
'Métodos': ['Sem Laplace', 'Com Laplace', 'Laplace + separação de emoji'],
'Porcentagem de acertos(%)': [por_acerts_sem_laplace,porcentagem_certos_sem_separar_emoji],
'Porcentagem de verdadeiros positivos(%)':[por_V_P_sem_laplace,Porcentagem_verdadeiros_positivo_sem_separar_emoji],
'Porcentagem de falsos positivos(%)':[por_F_P_sem_laplace,Porcentagem_falsos_positivos_sem_separar_emoji],
'Porcentagem de verdadeiros negativos(%)':[por_V_N_sem_laplace,Porcentagem_verdadeiros_negativos_sem_separar_emoji],
'Porcentagem de falsos negativos(%)':[por_F_N_sem_laplace,Porcentagem_falsos_negativos_sem_separar_emoji]
}
tabela_SEMxCOM = pd.DataFrame(data, columns=['Métodos', 'Porcentagem de acertos(%)','Porcentagem de verdadeiros positivos(%)','Porcentagem de falsos positivos(%)','Porcentagem de verdadeiros negativos(%)','Porcentagem de falsos negativos(%)'])

P_acertos=tabela_SEMxCOM['Porcentagem de acertos(%)']
P_V_P=tabela_SEMxCOM['Porcentagem de verdadeiros positivos(%)']
P_F_P=tabela_SEMxCOM['Porcentagem de falsos positivos(%)']
P_V_N=tabela_SEMxCOM['Porcentagem de verdadeiros negativos(%)']
P_F_N=tabela_SEMxCOM['Porcentagem de falsos negativos(%)']

ValueError: arrays must all be same length

In [30]:
plot = P_acertos.plot(kind='bar',title='Porcentagem de acertos(%)',figsize=(6, 6),color=('b','g'))
plt.ylabel('Porcentagem de acertos(%)')
plt.xlabel('SEM X COM')
plt.show()

plot = P_V_P.plot(kind='bar',title='Porcentagem de verdadeiros positivos(%)',figsize=(6, 6),color=('b','g'))
plt.ylabel('Porcentagem de verdadeiros positivos(%)')
plt.xlabel('SEM X COM')
plt.show()


plot = P_F_P.plot(kind='bar',title='Porcentagem de falsos positivos(%)',figsize=(6, 6),color=('b','g'))
plt.ylabel('Porcentagem de falsos positivos(%)')
plt.xlabel('SEM X COM')
plt.show()


plot = P_V_N.plot(kind='bar',title='Porcentagem de verdadeiros negativos(%)',figsize=(6, 6),color=('b','g'))
plt.ylabel('Porcentagem de verdadeiros negativos(%)')
plt.xlabel('SEM X COM')
plt.show()

plot = P_F_N.plot(kind='bar',title='PPorcentagem de falsos negativos(%)',figsize=(6, 6),color=('b','g'))
plt.ylabel('Porcentagem de falsos negativos(%)')
plt.xlabel('SEM X COM')
plt.show()

NameError: name 'P_acertos' is not defined

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**